In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Load the data
df_1= pd.read_csv('data1.csv')
df_2= pd.read_csv('data2.csv')
predcit_df = pd.read_csv('predicion.csv')

df_1.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
df_2.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [5]:
# unir los dos dataframes
df = pd.concat([df_1, df_2], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74250 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     74250 non-null  int64  
 1   amount_tsh             74250 non-null  float64
 2   date_recorded          74250 non-null  object 
 3   funder                 69743 non-null  object 
 4   gps_height             74250 non-null  int64  
 5   installer              69718 non-null  object 
 6   longitude              74250 non-null  float64
 7   latitude               74250 non-null  float64
 8   wpt_name               74248 non-null  object 
 9   num_private            74250 non-null  int64  
 10  basin                  74250 non-null  object 
 11  subvillage             73780 non-null  object 
 12  region                 74250 non-null  object 
 13  region_code            74250 non-null  int64  
 14  district_code          74250 non-null  int64  
 15  lga    

### Faltan datos en las columnas
#### funder,installer,wpt_name,subvillage,public_meeting,scheme_management,scheme_management,permit,

In [6]:
df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,74250.000000,74250.000000,74250.000000,74250.000000,7.425000e+04,74250.000000,74250.000000,74250.000000,74250.000000,74250.000000
mean,37124.500000,318.685704,665.667313,34.074262,-5.701771e+00,0.462330,15.265414,5.629077,180.750828,1298.463650
std,21434.273081,2906.762364,692.761033,6.572519,2.944969e+00,11.537879,17.508907,9.641636,471.086120,952.349375
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18562.250000,0.000000,0.000000,33.086819,-8.525675e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37124.500000,0.000000,364.000000,34.907475,-5.026540e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55686.750000,20.000000,1317.000000,37.181685,-3.325058e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74249.000000,350000.000000,2777.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [7]:
# ver que varables son categoricas y cuales numericas
print(f'Variables Categoricas{df.select_dtypes(include=['object']).columns}')
print(f'Varibales numericas{df.select_dtypes(include=['float64',
                          'int64']).columns}')

Variables CategoricasIndex(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')
Varibales numericasIndex(['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude',
       'num_private', 'region_code', 'district_code', 'population',
       'construction_year'],
      dtype='object')


In [8]:
#comprobar valores unicos en todas las columnas
for col in df.columns:
    print(f'{col}: {df[col].unique()}')

id: [69572  8776 34310 ... 28749 33492 68707]
amount_tsh: [6.00e+03 0.00e+00 2.50e+01 2.00e+01 2.00e+02 5.00e+02 5.00e+01 4.00e+03
 1.50e+03 6.00e+00 2.50e+02 1.00e+01 1.00e+03 1.00e+02 3.00e+01 2.00e+03
 4.00e+02 1.20e+03 4.00e+01 3.00e+02 2.50e+04 7.50e+02 5.00e+03 6.00e+02
 7.20e+03 2.40e+03 5.00e+00 3.60e+03 4.50e+02 4.00e+04 1.20e+04 3.00e+03
 7.00e+00 2.00e+04 2.80e+03 2.20e+03 7.00e+01 5.50e+03 1.00e+04 2.50e+03
 6.50e+03 5.50e+02 3.30e+01 8.00e+03 4.70e+03 7.00e+03 1.40e+04 1.30e+03
 1.00e+05 7.00e+02 1.00e+00 6.00e+01 3.50e+02 2.00e-01 3.50e+01 3.06e+02
 8.50e+03 1.17e+05 3.50e+03 5.20e+02 1.50e+01 6.30e+03 9.00e+03 1.50e+02
 1.20e+05 1.38e+05 3.50e+05 4.50e+03 1.30e+04 4.50e+04 2.00e+00 1.50e+04
 1.10e+04 5.00e+04 7.50e+03 1.63e+04 8.00e+02 1.60e+04 3.00e+04 5.30e+01
 5.40e+03 7.00e+04 2.50e+05 2.00e+05 2.60e+04 1.80e+04 2.60e+01 5.90e+02
 9.00e+02 9.00e+00 1.40e+03 1.70e+05 2.20e+02 3.80e+04 2.50e-01 1.20e+01
 6.00e+04 5.90e+01 3.50e+04 3.00e+00 2.55e+03 5.00e-01]
date_recor

### revisando los valores unicos de todas las variables se pueden hacer transformaciones en las variables:

#### waterpoint_type_group,waterpoint_type_group,source_class,source_type,source,quantity_group


In [9]:
# revisar valores nulos
df.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    4507
gps_height                   0
installer                 4532
longitude                    0
latitude                     0
wpt_name                     2
num_private                  0
basin                        0
subvillage                 470
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4155
recorded_by                  0
scheme_management         4847
scheme_name              36052
permit                    3793
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

#### Se hace la primera transformacion para calcualr la antiguedad del punto de agua y ver si es nuevo, viejo,antiguo o media y asi saber con mas presicion si por ejemplo llos puntos de agua contruidos en ciertos años tienen mas fallas

In [10]:
# Convertir a datetime
df["date_recorded"] = pd.to_datetime(df["date_recorded"])
df["year_recorded"] = df["date_recorded"].dt.year
df["month_recorded"] = df["date_recorded"].dt.month
df["day_recorded"] = df["date_recorded"].dt.day

predcit_df["date_recorded"] = pd.to_datetime(predcit_df["date_recorded"])
predcit_df["year_recorded"] = predcit_df["date_recorded"].dt.year
predcit_df["month_recorded"] = predcit_df["date_recorded"].dt.month
predcit_df["day_recorded"] = predcit_df["date_recorded"].dt.day

In [11]:
df["waterpoint_age"] = df["year_recorded"] - df["construction_year"]
df.loc[df["construction_year"] == 0, "waterpoint_age"] = df["waterpoint_age"].median()

predcit_df["waterpoint_age"] = predcit_df["year_recorded"] - predcit_df["construction_year"]
predcit_df.loc[predcit_df["construction_year"] == 0, "waterpoint_age"] = predcit_df["waterpoint_age"].median()

In [12]:
df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,month_recorded,day_recorded,waterpoint_age
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,3,14,12
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,6,3
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,2,25,4
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,1,28,27
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,7,13,26


### La segunda transformacion que podemos realizar en agrupar puntos de agua por proximidad esto nos va a permitir saber si los puntos de agua mas alejados del centro tienen mas o menos problemas

In [13]:
print(df[["longitude", "latitude", "gps_height"]].isnull().sum())

longitude     0
latitude      0
gps_height    0
dtype: int64


In [14]:
from sklearn.cluster import KMeans

# Aplicar K-Means con 10 clusters (ajustable)
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
df["location_cluster"] = kmeans.fit_predict(df[["longitude", "latitude"]])
predcit_df["location_cluster"] = kmeans.predict(predcit_df[["longitude", "latitude"]])

In [15]:
from geopy.distance import geodesic

# Crear un diccionario con el centro de cada región
region_centers = df.groupby("region")[["latitude", "longitude"]].median()
region_centers2= predcit_df.groupby("region")[["latitude", "longitude"]].median()

# Función para calcular la distancia geográfica
def calcular_distancia(row):
    centro = region_centers.loc[row["region"]]
    return geodesic((row["latitude"], row["longitude"]), (centro["latitude"], centro["longitude"])).km

def calcular_distancia2(row):
    centro = region_centers2.loc[row["region"]]
    return geodesic((row["latitude"], row["longitude"]), (centro["latitude"], centro["longitude"])).km

# Aplicar la función
df["distance_to_region_center"] = df.apply(calcular_distancia, axis=1)
predcit_df["distance_to_region_center"] = predcit_df.apply(calcular_distancia2, axis=1)

In [16]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,month_recorded,day_recorded,waterpoint_age,location_cluster,distance_to_region_center
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,spring,groundwater,communal standpipe,communal standpipe,2011,3,14,12,4,87.378976
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3,6,3,9,73.379187
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,dam,surface,communal standpipe multiple,communal standpipe,2013,2,25,4,6,206.064304
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,1,28,27,0,108.075822
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,7,13,26,2,26.536861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,river/lake,surface,communal standpipe,communal standpipe,2011,2,24,23,3,38.931083
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,shallow well,groundwater,hand pump,hand pump,2011,3,21,17,3,120.857430
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,dam,surface,communal standpipe,communal standpipe,2013,3,4,3,7,21.237615
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,river/lake,surface,communal standpipe,communal standpipe,2013,2,18,4,4,26.011852


In [17]:
predcit_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,month_recorded,day_recorded,waterpoint_age,location_cluster,distance_to_region_center
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,rainwater harvesting,surface,other,other,2013,2,4,1,7,36.392708
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,spring,groundwater,communal standpipe,communal standpipe,2013,2,4,13,6,6.620324
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,rainwater harvesting,surface,other,other,2013,2,1,3,7,26.880927
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,shallow well,groundwater,other,other,2013,1,22,26,0,121.189554
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,spring,groundwater,communal standpipe,communal standpipe,2013,3,27,13,4,64.869906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,39307,0.0,2011-02-24,Danida,34,Da,38.852669,-6.582841,Kwambwezi,0,...,river/lake,surface,communal standpipe,communal standpipe,2011,2,24,23,3,27.500315
14846,18990,1000.0,2011-03-21,Hiap,0,HIAP,37.451633,-5.350428,Bonde La Mkondoa,0,...,shallow well,groundwater,hand pump,hand pump,2011,3,21,17,3,118.626855
14847,28749,0.0,2013-03-04,NaN,1476,NaN,34.739804,-4.585587,Bwawani,0,...,dam,surface,communal standpipe,communal standpipe,2013,3,4,3,7,19.606547
14848,33492,0.0,2013-02-18,Germany,998,DWE,35.432732,-10.584159,Kwa John,0,...,river/lake,surface,communal standpipe,communal standpipe,2013,2,18,4,4,25.818823


#### La altura (gps_height) puede ser una variable importante si hay patrones de falla en áreas más altas o bajas.

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df["gps_height_scaled"] = scaler.fit_transform(df[["gps_height"]])
predcit_df["gps_height_scaled"] = scaler.transform(predcit_df[["gps_height"]])

### Tercera transformacion, convertimos las variables categoricas en numericas

In [21]:
threshold = 10  # Número mínimo de ocurrencias
# Seleccionar columnas categóricas
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols2 = predcit_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    counts = df[col].value_counts()
    mask = df[col].isin(counts[counts < threshold].index)
    df[col] = np.where(mask, 'Otros', df[col])

for col in cat_cols2:
    counts = predcit_df[col].value_counts()
    mask = predcit_df[col].isin(counts[counts < threshold].index)
    predcit_df[col] = np.where(mask, 'Otros', predcit_df[col])
    

import category_encoders as ce

# Aplicar Binary Encoding
encoder = ce.BinaryEncoder(cols=cat_cols)
encoder2 = ce.BinaryEncoder(cols=cat_cols2)
df_encoded = encoder.fit_transform(df)
df_encoded2 = encoder2.fit_transform(predcit_df)


pd.options.display.max_columns = None
print(df.head())  

      id  amount_tsh date_recorded   funder  gps_height     installer  \
0  69572      6000.0    2011-03-14    Roman        1390         Roman   
1   8776         0.0    2013-03-06  Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25    Otros         686  World vision   
3  67743         0.0    2013-01-28   Unicef         263        UNICEF   
4  19728         0.0    2011-07-13    Otros           0       Artisan   

   longitude   latitude  wpt_name  num_private                    basin  \
0  34.938093  -9.856322      none            0               Lake Nyasa   
1  34.698766  -2.147466  Zahanati            0            Lake Victoria   
2  37.460664  -3.821329     Otros            0                  Pangani   
3  38.486161 -11.155298     Otros            0  Ruvuma / Southern Coast   
4  31.130847  -1.825359   Shuleni            0            Lake Victoria   

   subvillage   region  region_code  district_code        lga        ward  \
0       Otros   Iringa           

In [22]:
# Rellenar valores nulos con la moda de cada columna
categorical_columns = ["funder", "installer", "scheme_name", "scheme_management",
                       "management", "payment", "water_quality", "quantity", "source", "waterpoint_type"]

for col in categorical_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)
    
for col in categorical_columns:
    predcit_df[col].fillna(predcit_df[col].mode()[0], inplace=True)

C:\Users\giova\AppData\Local\Temp\ipykernel_11100\4028474807.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\giova\AppData\Local\Temp\ipykernel_11100\4028474807.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [23]:
# Función para agrupar categorías poco frecuentes
def reducir_categorias(df, col, threshold=0.01):
    """Agrupa en 'Otros' las categorías que representan menos del threshold (%) de los datos"""
    counts = df[col].value_counts(normalize=True)
    rare_categories = counts[counts < threshold].index
    df[col] = df[col].replace(rare_categories, "Otros")
    return df

def reducir_categorias2(df, col, threshold=0.01):
    """Agrupa en 'Otros' las categorías que representan menos del threshold (%) de los datos"""
    counts = df[col].value_counts(normalize=True)
    rare_categories = counts[counts < threshold].index
    df[col] = df[col].replace(rare_categories, "Otros")
    return df

# Aplicar a las variables con muchas categorías
#for col in ["funder", "installer", "scheme_name"]:
    #df = reducir_categorias(df, col)

In [ ]:
#df = pd.get_dummies(df, columns=["water_quality", "quantity", "source", "waterpoint_type"], drop_first=True)

In [24]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Codificar las columnas categóricas
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

for column in categorical_columns:
    predcit_df[column] = label_encoder.fit_transform(predcit_df[column].astype(str))

In [25]:
# Contar las categorías únicas por columna categórica
category_counts = df[categorical_columns].nunique()
category_counts2 = predcit_df[categorical_columns].nunique()

# Imprimir las columnas con más de 20 categorías
columns_to_drop = category_counts[category_counts > 20].index
columns_to_drop2 = category_counts2[category_counts2 > 20].index
print(f"Eliminando las siguientes columnas con más de 20 categorías: {columns_to_drop}")
df = df.drop(columns=columns_to_drop)
predcit_df = predcit_df.drop(columns=columns_to_drop2)

Eliminando las siguientes columnas con más de 20 categorías: Index(['funder', 'installer', 'scheme_name'], dtype='object')


### Cuarta Trasformacion Manejo de outlayers

In [26]:
# Seleccionar columnas numéricas
numerical_columns = df.select_dtypes(include=["int64", "float64"]).columns
numerical_columns2 = predcit_df.select_dtypes(include=["int64", "float64"]).columns

# Función para identificar outliers
def detectar_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # Percentil 25
    Q3 = df[column].quantile(0.75)  # Percentil 75
    IQR = Q3 - Q1  # Rango intercuartil
    lower_bound = Q1 - 1.5 * IQR  # Límite inferior
    upper_bound = Q3 + 1.5 * IQR  # Límite superior
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]


# Detectar outliers en todas las columnas numéricas
outliers_dict = {col: detectar_outliers(df, col) for col in numerical_columns}
outliers_dict2 = {col: detectar_outliers(predcit_df, col) for col in numerical_columns2}

# Contar cuántos outliers hay por variable
outliers_count = {col: len(outliers_dict[col]) for col in numerical_columns}
print("Cantidad de outliers por variable:", outliers_count)

Cantidad de outliers por variable: {'id': 0, 'amount_tsh': 13971, 'gps_height': 0, 'longitude': 2269, 'latitude': 0, 'num_private': 951, 'region_code': 4480, 'district_code': 5237, 'population': 5502, 'scheme_management': 23486, 'construction_year': 0, 'management': 23626, 'payment': 0, 'water_quality': 10745, 'quantity': 975, 'source': 840, 'waterpoint_type': 0, 'waterpoint_age': 0, 'distance_to_region_center': 3356, 'gps_height_scaled': 0}


In [27]:
df_target = pd.read_csv('objetivo.csv')

df_final = pd.merge(df, df_target, on="id")
df_final

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,month_recorded,day_recorded,waterpoint_age,location_cluster,distance_to_region_center,gps_height_scaled,status_group
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Otros,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,6,False,1999,gravity,gravity,gravity,7,user-group,2,annually,6,good,1,enough,8,spring,groundwater,2,communal standpipe,2011,3,14,12,4,87.378976,0.516219,functional
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Otros,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,1,True,2010,gravity,gravity,gravity,11,user-group,0,never pay,6,good,2,insufficient,5,rainwater harvesting,surface,2,communal standpipe,2013,3,6,3,9,73.379187,0.519358,functional
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,Otros,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,6,True,2009,gravity,gravity,gravity,7,user-group,4,per bucket,6,good,1,enough,0,dam,surface,3,communal standpipe,2013,2,25,4,6,206.064304,0.270666,functional
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,Otros,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,6,True,1986,submersible,submersible,submersible,7,user-group,0,never pay,6,good,0,dry,3,borehole,groundwater,3,communal standpipe,2013,1,28,27,0,108.075822,0.123125,non functional
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Otros,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,6,True,0,gravity,gravity,gravity,1,other,0,never pay,6,good,3,seasonal,5,rainwater harvesting,surface,2,communal standpipe,2011,7,13,26,2,26.536861,0.031392,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,1210,37.169807,-3.253847,Otros,0,Pangani,Otros,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,9,True,1999,gravity,gravity,gravity,9,user-group,4,per bucket,6,good,1,enough,8,spring,groundwater,2,communal standpipe,2013,5,3,14,6,39.864443,0.453436,functional
59396,27263,4700.0,2011-05-07,1212,35.249991,-9.070629,Otros,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,6,True,1996,gravity,gravity,gravity,7,user-group,2,annually,6,good,1,enough,6,river/lake,surface,2,communal standpipe,2011,5,7,15,4,52.187588,0.454133,functional
59397,37057,0.0,2011-04-11,0,34.017087,-8.750434,Mashine,0,Rufiji,Otros,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,6,False,0,swn 80,swn 80,handpump,7,user-group,3,monthly,1,fluoride,1,enough,3,borehole,groundwater,4,hand pump,2011,4,11,26,8,60.056744,0.031392,functional
59398,31282,0.0,2011-03-08,0,35.861315,-6.378573,Otros,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,6,True,0,nira/tanira,nira/tanira,handpump,7,user-group,0,never pay,6,good,2,insufficient,7,shallow well,groundwater,4,hand pump,2011,3,8,26,7,31.897328,0.031392,functional


In [28]:
#ver las variables q no sean numericas
print(f'Variables Categoricas{df_final.select_dtypes(include=["object"]).columns}')

Variables CategoricasIndex(['wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward',
       'public_meeting', 'recorded_by', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management_group',
       'payment_type', 'quality_group', 'quantity_group', 'source_type',
       'source_class', 'waterpoint_type_group', 'status_group'],
      dtype='object')


In [29]:
df_final = df_final.drop(columns=['recorded_by','date_recorded','subvillage'])
predcit_df = predcit_df.drop(columns=['recorded_by','date_recorded','subvillage'])


In [30]:
# Codificar la variable objetivo 'status_group'
le = LabelEncoder()
df_final['status_group'] = le.fit_transform(df_final['status_group'])

# Realizar One-Hot Encoding para las variables con muchas categorías
df_final = pd.get_dummies(df_final, columns=['wpt_name', 'basin','region', 'lga', 'ward', 
                                              'extraction_type', 'extraction_type_group', 'extraction_type_class', 
                                              'management_group', 'payment_type', 'quality_group', 'quantity_group', 
                                              'source_type', 'source_class', 'waterpoint_type_group'], drop_first=True)

predcit_df = pd.get_dummies(predcit_df, columns=['wpt_name', 'basin', 'region', 'lga', 'ward', 
                                              'extraction_type', 'extraction_type_group', 'extraction_type_class', 
                                              'management_group', 'payment_type', 'quality_group', 'quantity_group', 
                                              'source_type', 'source_class', 'waterpoint_type_group'], drop_first=True)

# Realizar Label Encoding para las variables binarias
le = LabelEncoder()
df_final['public_meeting'] = le.fit_transform(df_final['public_meeting'])
df_final['permit'] = le.fit_transform(df_final['permit'])

predcit_df['public_meeting'] = le.fit_transform(predcit_df['public_meeting'])
predcit_df['permit'] = le.fit_transform(predcit_df['permit'])

In [31]:
predcit_df

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,public_meeting,scheme_management,permit,construction_year,management,payment,water_quality,quantity,source,waterpoint_type,year_recorded,month_recorded,day_recorded,waterpoint_age,location_cluster,distance_to_region_center,gps_height_scaled,wpt_name_Barabarani,wpt_name_Bombani,wpt_name_Bondeni,wpt_name_Bwawani,wpt_name_Ccm,wpt_name_Center,wpt_name_Dispensary,wpt_name_Kanisani,wpt_name_Kijiweni,wpt_name_Kilabuni,wpt_name_Kisimani,wpt_name_Kituo Cha Afya,wpt_name_Madukani,wpt_name_Maendeleo,wpt_name_Mahakamani,wpt_name_Majengo,wpt_name_Mashineni,wpt_name_Mbugani,wpt_name_Miembeni,wpt_name_Mkombozi,wpt_name_Mkuyuni,wpt_name_Mkwajuni,wpt_name_Mlimani,wpt_name_Msikitini,wpt_name_Mtakuja,wpt_name_Muungano,wpt_name_Mwembeni,wpt_name_Ofisi Ya Kijiji,wpt_name_Ofisini,wpt_name_Otros,wpt_name_Rc Church,wpt_name_School,wpt_name_Secondary,wpt_name_Sekondari,wpt_name_Shule,wpt_name_Shule Ya Msingi,wpt_name_Shuleni,wpt_name_Shuleni Sekondari,wpt_name_Sokoni,wpt_name_Songambele,wpt_name_Tenkini,wpt_name_Tumaini,wpt_name_Umoja,wpt_name_Upendo,wpt_name_Uwanjani,wpt_name_Zahanati,wpt_name_none,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,lga_Arusha Urban,lga_Babati,lga_Bagamoyo,lga_Bahi,lga_Bariadi,lga_Biharamulo,lga_Bukoba Rural,lga_Bukoba Urban,lga_Bukombe,lga_Bunda,lga_Chamwino,lga_Chato,lga_Chunya,lga_Dodoma Urban,lga_Geita,lga_Hai,lga_Hanang,lga_Handeni,lga_Igunga,lga_Ilala,lga_Ileje,lga_Ilemela,lga_Iramba,lga_Iringa Rural,lga_Kahama,lga_Karagwe,lga_Karatu,lga_Kasulu,lga_Kibaha,lga_Kibondo,lga_Kigoma Rural,lga_Kigoma Urban,lga_Kilindi,lga_Kilolo,lga_Kilombero,lga_Kilosa,lga_Kilwa,lga_Kinondoni,lga_Kisarawe,lga_Kishapu,lga_Kiteto,lga_Kondoa,lga_Kongwa,lga_Korogwe,lga_Kwimba,lga_Kyela,lga_Lindi Rural,lga_Liwale,lga_Longido,lga_Ludewa,lga_Lushoto,lga_Mafia,lga_Magu,lga_Makete,lga_Manyoni,lga_Masasi,lga_Maswa,lga_Mbarali,lga_Mbeya Rural,lga_Mbinga,lga_Mbozi,lga_Mbulu,lga_Meatu,lga_Meru,lga_Misenyi,lga_Missungwi,lga_Mkinga,lga_Mkuranga,lga_Monduli,lga_Morogoro Rural,lga_Morogoro Urban,lga_Moshi Rural,lga_Moshi Urban,lga_Mpanda,lga_Mpwapwa,lga_Mtwara Rural,lga_Mtwara Urban,lga_Mufindi,lga_Muheza,lga_Muleba,lga_Musoma Rural,lga_Mvomero,lga_Mwanga,lga_Nachingwea,lga_Namtumbo,lga_Nanyumbu,lga_Newala,lga_Ngara,lga_Ngorongoro,lga_Njombe,lga_Nkasi,lga_Nzega,lga_Otros,lga_Pangani,lga_Rombo,lga_Rorya,lga_Ruangwa,lga_Rufiji,lga_Rungwe,lga_Same,lga_Sengerema,lga_Serengeti,lga_Shinyanga Rural,lga_Shinyanga Urban,lga_Siha,lga_Sikonge,lga_Simanjiro,lga_Singida Rural,lga_Singida Urban,lga_Songea Rural,lga_Songea Urban,lga_Sumbawanga Rural,lga_Sumbawanga Urban,lga_Tabora Urban,lga_Tandahimba,lga_Tanga,lga_Tarime,lga_Temeke,lga_Tunduru,lga_Ukerewe,lga_Ulanga,lga_Urambo,lga_Uyui,ward_Berega,ward_Biharamulo Urban,ward_Bitale,ward_Boma la ng'ombe,ward_Buchambi,ward_Bugarama,ward_Bugoro,ward_Buhendangabo,ward_Bujashi,ward_Bujonde,ward_Bumera,ward_Bunamhala,ward_Bungu,ward_Bunyambo,ward_Busagara,ward_Busawe,ward_Busilili,ward_Busole,ward_Buterankuzi,ward_Bwakila Chini,ward_Bwambo,ward_Chakwale,ward_Chala,ward_Chalinze,ward_Chambo,ward_Chamkoroma,ward_Chanika,ward_Chanzuru,ward_Chela,ward_Chigungu,ward_Chimala,ward_Chinamili,ward_Chitete,ward_Chiugutwa,ward_Choma,ward_Chumo,ward_Dareda,ward_Didia,ward_Diongoya,ward_Doma,ward_Dongobesh,ward_Dumila,ward_Dutwa,ward_Endabash,ward_Endasak,ward_Engarenaibor,ward_Gwanumpu,ward_Haidom,ward_Halungu,ward_Hedaru,ward_Hembeti,ward_Hombolo,ward_Hungumalwa,ward_Ibadakuli,ward_Idodi,ward_Ifakara,ward_Ifunda,ward_Igalukilo,ward_Igamba,ward_Igongolo,wa

In [32]:
# Separar las variables independientes (X) y la variable dependiente (y)
X = df_final.drop(columns=["status_group"])  # Reemplaza "target_column" con el nombre de tu columna objetivo
y = df_final["status_group"]  # Variable objetivo

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf.fit(X_train, y_train)

# Realizar predicciones
y_pred_rf = rf.predict(X_test)

# Evaluar el modelo
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8003928170594837
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.88      0.84      9724
           1       0.50      0.34      0.40      1293
           2       0.83      0.78      0.80      6803

    accuracy                           0.80     17820
   macro avg       0.71      0.66      0.68     17820
weighted avg       0.79      0.80      0.79     17820



In [34]:
df_final


id  amount_tsh  gps_height  longitude   latitude  num_private  \
0      69572      6000.0        1390  34.938093  -9.856322            0   
1       8776         0.0        1399  34.698766  -2.147466            0   
2      34310        25.0         686  37.460664  -3.821329            0   
3      67743         0.0         263  38.486161 -11.155298            0   
4      19728         0.0           0  31.130847  -1.825359            0   
...      ...         ...         ...        ...        ...          ...   
59395  60739        10.0        1210  37.169807  -3.253847            0   
59396  27263      4700.0        1212  35.249991  -9.070629            0   
59397  37057         0.0           0  34.017087  -8.750434            0   
59398  31282         0.0           0  35.861315  -6.378573            0   
59399  26348         0.0         191  38.104048  -6.747464            0   

       region_code  district_code  population  public_meeting  \
0               11              5         109               1   
1               20              2         280               2   
2               21              4         250               1   
3               90             63          58               1   
4               18              1           0               1   
...            ...            ...         ...             ...   
59395            3              5         125               1   
59396           11              4          56               1   
59397           12              7           0               1   
59398            1              4           0               1   
59399            5              2         150               1   

       scheme_management  permit  construction_year  management  payment  \
0                      6       0               1999           7        2   
1                      1       1               2010          11        0   
2                      6       1               2009           7        4   
3                      6       1               1986           7        0   
4                      6       1                  0           1        0   
...                  ...     ...                ...         ...      ...   
59395                  9       1               1999           9        4   
59396                  6       1               1996           7        2   
59397                  6       0                  0           7        3   
59398                  6       1                  0           7        0   
59399                  6       1               2002           7        5   

       water_quality  quantity  source  waterpoint_type  year_recorded  \
0                  6         1       8                2           2011   
1                  6         2       5                2           2013   
2                  6         1       0                3           2013   
3                  6         0       3                3           2013   
4                  6         3       5                2           2011   
...              ...       ...     ...              ...            ...   
59395              6         1       8                2           2013   
59396              6         1       6                2           2011   
59397              1         1       3                4           2011   
59398              6         2       7                4           2011   
59399              4         1       7                4           2011   

       month_recorded  day_recorded  waterpoint_age  location_cluster  \
0                   3            14              12                 4   
1                   3             6               3                 9   
2                   2            25               4                 6   
3                   1            28              27                 0   
4                   7            13              26                 2   
...               ...           ...             ...               ...   
59395          

In [35]:
predcit_df

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,public_meeting,scheme_management,permit,construction_year,management,payment,water_quality,quantity,source,waterpoint_type,year_recorded,month_recorded,day_recorded,waterpoint_age,location_cluster,distance_to_region_center,gps_height_scaled,wpt_name_Barabarani,wpt_name_Bombani,wpt_name_Bondeni,wpt_name_Bwawani,wpt_name_Ccm,wpt_name_Center,wpt_name_Dispensary,wpt_name_Kanisani,wpt_name_Kijiweni,wpt_name_Kilabuni,wpt_name_Kisimani,wpt_name_Kituo Cha Afya,wpt_name_Madukani,wpt_name_Maendeleo,wpt_name_Mahakamani,wpt_name_Majengo,wpt_name_Mashineni,wpt_name_Mbugani,wpt_name_Miembeni,wpt_name_Mkombozi,wpt_name_Mkuyuni,wpt_name_Mkwajuni,wpt_name_Mlimani,wpt_name_Msikitini,wpt_name_Mtakuja,wpt_name_Muungano,wpt_name_Mwembeni,wpt_name_Ofisi Ya Kijiji,wpt_name_Ofisini,wpt_name_Otros,wpt_name_Rc Church,wpt_name_School,wpt_name_Secondary,wpt_name_Sekondari,wpt_name_Shule,wpt_name_Shule Ya Msingi,wpt_name_Shuleni,wpt_name_Shuleni Sekondari,wpt_name_Sokoni,wpt_name_Songambele,wpt_name_Tenkini,wpt_name_Tumaini,wpt_name_Umoja,wpt_name_Upendo,wpt_name_Uwanjani,wpt_name_Zahanati,wpt_name_none,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,basin_Rufiji,basin_Ruvuma / Southern Coast,basin_Wami / Ruvu,region_Dar es Salaam,region_Dodoma,region_Iringa,region_Kagera,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Mtwara,region_Mwanza,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Singida,region_Tabora,region_Tanga,lga_Arusha Urban,lga_Babati,lga_Bagamoyo,lga_Bahi,lga_Bariadi,lga_Biharamulo,lga_Bukoba Rural,lga_Bukoba Urban,lga_Bukombe,lga_Bunda,lga_Chamwino,lga_Chato,lga_Chunya,lga_Dodoma Urban,lga_Geita,lga_Hai,lga_Hanang,lga_Handeni,lga_Igunga,lga_Ilala,lga_Ileje,lga_Ilemela,lga_Iramba,lga_Iringa Rural,lga_Kahama,lga_Karagwe,lga_Karatu,lga_Kasulu,lga_Kibaha,lga_Kibondo,lga_Kigoma Rural,lga_Kigoma Urban,lga_Kilindi,lga_Kilolo,lga_Kilombero,lga_Kilosa,lga_Kilwa,lga_Kinondoni,lga_Kisarawe,lga_Kishapu,lga_Kiteto,lga_Kondoa,lga_Kongwa,lga_Korogwe,lga_Kwimba,lga_Kyela,lga_Lindi Rural,lga_Liwale,lga_Longido,lga_Ludewa,lga_Lushoto,lga_Mafia,lga_Magu,lga_Makete,lga_Manyoni,lga_Masasi,lga_Maswa,lga_Mbarali,lga_Mbeya Rural,lga_Mbinga,lga_Mbozi,lga_Mbulu,lga_Meatu,lga_Meru,lga_Misenyi,lga_Missungwi,lga_Mkinga,lga_Mkuranga,lga_Monduli,lga_Morogoro Rural,lga_Morogoro Urban,lga_Moshi Rural,lga_Moshi Urban,lga_Mpanda,lga_Mpwapwa,lga_Mtwara Rural,lga_Mtwara Urban,lga_Mufindi,lga_Muheza,lga_Muleba,lga_Musoma Rural,lga_Mvomero,lga_Mwanga,lga_Nachingwea,lga_Namtumbo,lga_Nanyumbu,lga_Newala,lga_Ngara,lga_Ngorongoro,lga_Njombe,lga_Nkasi,lga_Nzega,lga_Otros,lga_Pangani,lga_Rombo,lga_Rorya,lga_Ruangwa,lga_Rufiji,lga_Rungwe,lga_Same,lga_Sengerema,lga_Serengeti,lga_Shinyanga Rural,lga_Shinyanga Urban,lga_Siha,lga_Sikonge,lga_Simanjiro,lga_Singida Rural,lga_Singida Urban,lga_Songea Rural,lga_Songea Urban,lga_Sumbawanga Rural,lga_Sumbawanga Urban,lga_Tabora Urban,lga_Tandahimba,lga_Tanga,lga_Tarime,lga_Temeke,lga_Tunduru,lga_Ukerewe,lga_Ulanga,lga_Urambo,lga_Uyui,ward_Berega,ward_Biharamulo Urban,ward_Bitale,ward_Boma la ng'ombe,ward_Buchambi,ward_Bugarama,ward_Bugoro,ward_Buhendangabo,ward_Bujashi,ward_Bujonde,ward_Bumera,ward_Bunamhala,ward_Bungu,ward_Bunyambo,ward_Busagara,ward_Busawe,ward_Busilili,ward_Busole,ward_Buterankuzi,ward_Bwakila Chini,ward_Bwambo,ward_Chakwale,ward_Chala,ward_Chalinze,ward_Chambo,ward_Chamkoroma,ward_Chanika,ward_Chanzuru,ward_Chela,ward_Chigungu,ward_Chimala,ward_Chinamili,ward_Chitete,ward_Chiugutwa,ward_Choma,ward_Chumo,ward_Dareda,ward_Didia,ward_Diongoya,ward_Doma,ward_Dongobesh,ward_Dumila,ward_Dutwa,ward_Endabash,ward_Endasak,ward_Engarenaibor,ward_Gwanumpu,ward_Haidom,ward_Halungu,ward_Hedaru,ward_Hembeti,ward_Hombolo,ward_Hungumalwa,ward_Ibadakuli,ward_Idodi,ward_Ifakara,ward_Ifunda,ward_Igalukilo,ward_Igamba,ward_Igongolo,wa

In [36]:
# ver que columnas tiene el df_final y el predcit_df y quitar las columnas de df_final que no esten en el predcit_df
df_final_columns = set(df_final.columns)
predcit_df_columns = set(predcit_df.columns)
columns_to_drop = df_final_columns - predcit_df_columns 
predcit_df = predcit_df.drop(columns=columns_to_drop)

# Asegurarse de que las columnas estén en el mismo orden




KeyError: '[\'ward_Mwakilyambiti\', \'wpt_name_Kwa Ally\', \'ward_Sepeko\', \'wpt_name_Mjini\', \'ward_Songa\', \'wpt_name_Makaburini\', \'wpt_name_Kwa Daudi\', \'wpt_name_Stendi\', \'wpt_name_Kwa Mzee Shabani\', \'ward_Sadani\', \'ward_Ussoke\', \'wpt_name_Makiungu Hospital\', \'wpt_name_Office Ya Kijiji\', \'ward_Vibaoni\', \'ward_Ngima\', \'ward_Nsunga\', \'wpt_name_Nasula Primary School\', \'ward_Marumbo\', \'ward_Rangwi\', \'wpt_name_Kota\', \'wpt_name_Omari\', \'ward_Kisorya\', \'ward_Kibamba\', \'ward_Sima\', \'ward_Mahuninga\', \'ward_Chato\', \'ward_Mtitaa\', \'wpt_name_Musa\', \'ward_Solwa\', \'ward_Muriaza\', \'ward_Misechela\', \'ward_Iwungilo\', \'ward_Masoko\', \'ward_Makiidi\', \'ward_Sopa\', \'ward_Kitobo\', \'ward_Mateves\', \'ward_Chalangwa\', \'ward_Njinjo\', \'ward_Usisya\', \'ward_Olbalbal\', \'ward_Isikizya\', \'ward_Kilema kaskazini\', \'ward_Kirya\', \'ward_Nahukahuka\', \'wpt_name_Hospital\', \'wpt_name_Tupendane\', \'ward_Kilimatinde\', \'ward_Sangabuye\', \'w

In [ ]:
predcit_df

In [33]:
X_new = predcit_df.copy()
predictions = rf.predict(X_new)
predcit_df["prediccion"]= predictions
predcit_df.to_csv('resultados_predicciones.csv', index=False)
print("\nPrimeras 5 predicciones:")
print(predcit_df[['id', 'prediccion']].head())

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- extraction_type_climax
- lga_Lindi Urban
- ward_Akheri
- ward_Arash
- ward_Arri
- ...
